In [10]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import trino
import sys, os, time, datetime
from datetime import date
import argparse, platform, csv
from requests.auth import HTTPBasicAuth
import requests.packages.urllib3 as urllib3
from trino.dbapi import connect
urllib3.disable_warnings()
import matplotlib.pyplot as plt
#from statsmodels.tsa.stattools import adfuller
#from arch.unitroot import bds
import trino
import pandas as pd
from datetime import date
import time


In [11]:


TRINO_LARGE_CONN=trino.dbapi.connect(
    host='trino-nce.corp.chartercom.com',
    port=443,
    user='P3185033',
    catalog='minio_prod',
    schema='proactive',
    http_scheme='https',
    verify=False,
auth=trino.auth.BasicAuthentication("P3185033", "Sagittarius@97"),
)


def trino_query_runner(query):
    TRINO_LARGE_CONN._http_session.verify = False
    cd = date.today()
    start_time = time.time()
    cur_trino = TRINO_LARGE_CONN.cursor()
    cur_trino.execute("SET SESSION minio_prod.insert_existing_partitions_behavior = 'overwrite'")
    rows = cur_trino.fetchall()
    cur_trino.execute(query)
    df = pd.DataFrame(cur_trino.fetchall(), columns=[i[0] for i in cur_trino.description])
    print(rows)
    print("Load Time: --- %s seconds ---" % (time.time() - start_time))
    return df




In [12]:
sql = """
select * from minio_prod.proactive.carolina_demographics_analysis
"""
print(sql)
df = trino_query_runner(sql)
df.head(5)


select * from minio_prod.proactive.carolina_demographics_analysis

[]
Load Time: --- 64.0022406578064 seconds ---


,account,no_of_children,home_age_bracket,no_married_residents,home_occupancy,owner_rental_info,length_of_residence,home_value,dwell_type,education_backgnd,ethnicity_code,ethinicity_group_code,language_spoken,county_size_code,family_composed_code,maleadulthh,femaleadulthh,occupationcd,lgcycorp,prizmsocialgroupslvl1_desc
0,8349300821754873,,25-34 YEARS OLD,AT LEAST 1 SINGLE,ONE PERSON,PROBABLE RENTER,3-5 YRS,,2,UNKNOWN,Hispanic,Hispanic,ENGLISH,B,ONE PERSON (MALE HH) WITH NO CHILD,1,,00,T,The Affluentials
1,8349100300659274,,18-24 YEARS OLD,AT LEAST 1 SINGLE,ONE PERSON,PROBABLE RENTER,7-12 MOS.,,1,COLLEGE,Western European,Caucasian/Other,ENGLISH,B,ONE PERSON (MALE HH) WITH NO CHILD,1,,00,T,The Affluentials
2,8351300180154678,ONE CHILD,25-34 YEARS OLD,AT LEAST 1 MARRIED,TWO PERSONS,DEFINITE OWNER,6-10 YRS,$95000,1,HIGH SCHOOL,Italian,Caucasian/Other,ENGLISH,D,ONE PERSON (FEMALE HH) WITH NO CHILD,1,,00,C,Rustic Living
3,8349300700807784,ONE CHILD,35-44 YEARS OLD,AT LEAST 1 SINGLE,TWO PERSONS,DEFINITE OWNER,0-6 MOS.,$1201000,1,COLLEGE,Western European,Caucasian/Other,ENGLISH,B,"1 MALE,1 FEMALE - MARITAL STATUS UNK, NO CHILD",,,00,T,The Affluentials
4,8349200021448468,,25-34 YEARS OLD,AT LEAST 1 SINGLE,ONE PERSON,PROBABLE RENTER,3-5 YRS,,2,SOME COLLEGE,African American,African American,ENGLISH,B,ONE PERSON (MALE HH) WITH NO CHILD,1,,00,T,City Centers


In [14]:
df.isna().sum()

account                     0
ntf_cnt                     0
inhome_cnt                  0
wholehome_cnt               0
partialhome_cnt             0
drop_cnt                    0
plant_cnt                   0
high_cnt                    0
medium_cnt                  0
low_cnt                     0
dsrx_cnt                    0
dsuncorrectable_cnt         0
dssnr_cnt                   0
hpmt_hpmtm_cnt              0
usripples_cnt               0
usuncorrectable_cnt         0
ustx_cnt                    0
ussnr_cnt                   0
highrx_cnt                  0
lowtx_cnt                   0
int_severity1               0
int_severity2               0
int_severity3               0
int_severity4               0
int_severity5               0
int_severity6               0
int_severity7               0
int_severity8               0
int_severity9               0
int_severity10              0
int_severity11              0
int_severity12              0
int_severity13              0
int_severi

In [4]:
data= df.fillna(0)

In [5]:
data_df = data[['account','ntf_cnt', 'inhome_cnt', 'wholehome_cnt', 'partialhome_cnt',
       'drop_cnt', 'plant_cnt', 'high_cnt', 'medium_cnt', 'low_cnt','dsrx_cnt', 'dsuncorrectable_cnt',
        'dssnr_cnt', 'hpmt_hpmtm_cnt',
       'usripples_cnt', 'usuncorrectable_cnt', 'ustx_cnt', 'ussnr_cnt',
       'highrx_cnt', 'lowtx_cnt' ,'int_severity1', 'int_severity2',
       'int_severity3', 'int_severity4', 'int_severity5', 'int_severity6',
       'int_severity7', 'int_severity8', 'int_severity9', 'int_severity10',
       'int_severity11', 'int_severity12', 'int_severity13', 'int_severity14',
       'int_severity15', 'int_severity16', 'int_severity17', 'int_severity18',
       'int_severity19', 'int_severity20', 'int_severity21', 'int_severity22',
       'int_severity23', 'int_severity24', 'etds_past1yearcnt', 
        'download_ingb', 'upload_ingb','ecafs_cnt', 'num_devices','predictor']]

In [17]:
data_df

,account,ntf_cnt,inhome_cnt,wholehome_cnt,partialhome_cnt,drop_cnt,plant_cnt,high_cnt,medium_cnt,low_cnt,...,int_severity21,int_severity22,int_severity23,int_severity24,etds_past1yearcnt,download_ingb,upload_ingb,ecafs_cnt,num_devices,predictor
0,8347401110324147,89,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,252.80,58.05,0.0,1,0
1,8245112000167867,203,0,115,0,0,0,3,114,0,...,0,0,0,0,0.0,2401.08,215.49,0.0,1,0
2,8345781160019016,417,46,0,0,0,0,12,34,4,...,0,0,0,0,0.0,9770.24,393.00,0.0,1,0
3,8352305500262480,460,0,0,0,0,0,0,1,0,...,0,0,0,0,0.0,4288.84,809.74,0.0,1,0
4,8352305730035912,462,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,6346.49,1721.82,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209635,8348100022353876,457,0,0,0,0,9,9,0,0,...,0,0,0,0,0.0,930.29,19.10,0.0,1,0
209636,8348100023071022,325,117,0,16,0,6,76,65,0,...,0,0,0,0,0.0,159.95,11.91,0.0,1,0
209637,8348100023228077,414,45,0,6,0,1,19,2,31,...,1,0,0,27,0.0,4346.27,205.34,0.0,1,0
209638,8348100023288204,196,210,0,3,0,0,7,161,63,...,0,0,0,0,0.0,250.21,26.46,0.0,1,0


In [6]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN

# Split the dataset into training and testing sets
X_train, X_test_actual, y_train, y_test_actual = train_test_split(data_df.drop('predictor', axis=1), data_df['predictor'], test_size=0.2, random_state=42)



# Use ADASYN to oversample the minority class
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_train.iloc[:,1:], y_train)

In [10]:
y_resampled.value_counts()

0    157436
1    153319
Name: predictor, dtype: int64

In [17]:
ray status

SyntaxError: invalid syntax (397495004.py, line 1)

In [15]:
import ray
ip_address = "10.28.68.121:52365"
ray.init(address = ip_address, ignore_reinit_error=True)

2023-06-25 20:28:38,763	INFO worker.py:1452 -- Connecting to existing Ray cluster at address: 10.28.68.121:52365...
2023-06-25 20:28:44,478	ERROR utils.py:1390 -- Failed to connect to GCS. Please check `gcs_server.out` for more details.
2023-06-25 20:28:44,479	WARNING utils.py:1396 -- Unable to connect to GCS at 10.28.68.121:52365. Check that (1) Ray GCS with matching version started successfully at the specified address, and (2) there is no firewall setting preventing access.
2023-06-25 20:28:52,074	ERROR utils.py:1390 -- Failed to connect to GCS. Please check `gcs_server.out` for more details.
2023-06-25 20:28:52,075	WARNING utils.py:1396 -- Unable to connect to GCS at 10.28.68.121:52365. Check that (1) Ray GCS with matching version started successfully at the specified address, and (2) there is no firewall setting preventing access.
2023-06-25 20:28:59,689	ERROR utils.py:1390 -- Failed to connect to GCS. Please check `gcs_server.out` for more details.
2023-06-25 20:28:59,690	WARNING

ConnectionError: Could not read 'session_name' from GCS. Did GCS start successfully?

In [7]:
import ray
#ip_address = "10.28.68.121:52365"
ray.init()

2023-06-29 10:00:44,412	INFO worker.py:1636 -- Started a local Ray instance.


Python version:,3.10.9
Ray version:,2.5.1


In [19]:
import ray

#ray.init(address='auto')
@ray.remote
def compute_intensive_task():
    # Perform a computationally intensive operation
    result = 0
    for i in range(10**8):
        result += i
    return result
results = []
for _ in range(10):
    result_id = compute_intensive_task.remote()
    results.append(result_id)
results = []
for _ in range(10):
    result_id = compute_intensive_task.remote()
    results.append(result_id)
output = ray.get(results)
print(output)

[4999999950000000, 4999999950000000, 4999999950000000, 4999999950000000, 4999999950000000, 4999999950000000, 4999999950000000, 4999999950000000, 4999999950000000, 4999999950000000]


In [23]:
!pip install -U ray

In [24]:
!pip install -U ray[tune]

In [1]:
from ray import tune
import ray
#ip_address = "10.28.68.121:52365"
ray.init()

def objective(config):  # ①
    score = config["a"] ** 2 + config["b"]
    return {"score": score}


search_space = {  # ②
    "a": tune.grid_search([0.001, 0.01, 0.1, 1.0]),
    "b": tune.choice([1, 2, 3]),
}

tuner = tune.Tuner(objective, param_space=search_space)  # ③

results = tuner.fit()
print(results.get_best_result(metric="score", mode="min").config)

2023-06-26 23:22:24,958	INFO worker.py:1636 -- Started a local Ray instance.


Trial name,date,done,experiment_tag,hostname,iterations_since_restore,node_ip,pid,score,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
objective_3bbec_00000,2023-06-26_23-22-43,True,"0_a=0.0010,b=1",CORTOPSAL8XDPSQ,1,127.0.0.1,27572,1,0.00100017,0.00100017,0.00100017,1687839763,1,3bbec_00000
objective_3bbec_00001,2023-06-26_23-22-43,True,"1_a=0.0100,b=1",CORTOPSAL8XDPSQ,1,127.0.0.1,10236,1.0001,0.000999689,0.000999689,0.000999689,1687839763,1,3bbec_00001
objective_3bbec_00002,2023-06-26_23-22-43,True,"2_a=0.1000,b=2",CORTOPSAL8XDPSQ,1,127.0.0.1,27328,2.01,0.00100064,0.00100064,0.00100064,1687839763,1,3bbec_00002
objective_3bbec_00003,2023-06-26_23-22-43,True,"3_a=1.0000,b=3",CORTOPSAL8XDPSQ,1,127.0.0.1,20692,4,0.00599527,0.00599527,0.00599527,1687839763,1,3bbec_00003


2023-06-26 23:22:43,690	WARNING tune.py:1085 -- Trial Runner checkpointing failed: Sync process failed: GetFileInfo() yielded path 'C:/Users/P3185033/ray_results/objective_2023-06-26_23-22-31/basic-variant-state-2023-06-26_23-22-33.json', which is outside base dir 'C:\Users\P3185033\ray_results\objective_2023-06-26_23-22-31'
2023-06-26 23:22:43,708	INFO tune.py:1111 -- Total run time: 12.25 seconds (9.82 seconds for the tuning loop).


{'a': 0.001, 'b': 1}


In [9]:
import ray
ray.shutdown()
RAY_ENABLE_WINDOWS_OR_OSX_CLUSTER=1
ray.init("ray://ncepntrinoa0001.corp.chartercom.com:10001")

C:\Users\P3185033\AppData\Local\anaconda3\lib\site-packages\ray\util\client\worker.py:253: UserWarning: Ray Client connection timed out. Ensure that the Ray Client port on the head node is reachable from your local machine. See https://docs.ray.io/en/latest/cluster/ray-client.html#step-2-check-ports for more information.
  warnings.warn(


ConnectionError: ray client connection timeout

In [8]:

from ray.tune.schedulers import HyperBandScheduler
import xgboost as xgb
from ray import tune
import ray
#ip_address = "10.28.68.121:52365"
# ray.init()


# Shutdown Ray
ray.shutdown()

if not ray.is_initialized():
    ray.init(ignore_reinit_error=True)

# Define your training function
def train_xgboost(config):
    # Load your dataset
    # Assuming you have features (X) and labels (y) as numpy arrays
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
    train_data = (ray.put(X_train), ray.put(y_train))  # Put large objects in Ray object store
    test_data = (ray.put(X_test), ray.put(y_test))  # Put large objects in Ray object store
    # Configure XGBoost with the given hyperparameters
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=config['n_estimators'],
        max_depth=config['max_depth'],
        learning_rate=config['learning_rate'],
        subsample=config['subsample'],
        # Add more hyperparameters as needed
    )

    # Train the model
    model.fit(X_train, y_train)

    # Evaluate the model
    accuracy = model.score(X_test, y_test)

    # Return the accuracy as the result
    return accuracy

# Define the hyperparameter search space
search_space = {
    'n_estimators': tune.choice([100, 200, 300]),
    'max_depth': tune.choice([3, 6, 9]),
    'learning_rate': tune.loguniform(0.001, 0.1),
    'subsample': tune.uniform(0.5, 1.0),
    # Add more hyperparameters and their search spaces as needed
}


# Initialize Ray
#desired_memory = 10737418240 # 10GB
#object_store_memory=desired_memory

# ray.init(ignore_reinit_error=True)
# #num_cpus=4, num_workers=2


# Define the hyperparameter search configuration
config = {
    'num_samples': 10,  # Number of trials
    'config': search_space,
    'stop': {'training_iteration': 100},  # Specify the stopping criteria
}
# Configure the scheduler (optional)
scheduler = HyperBandScheduler(
    time_attr='training_iteration',
    metric = 'mean_accuracy',
    mode = 'max'
    #max_t=100,
    #grace_period=10
)

# Run the hyperparameter search
analysis = tune.run(
    train_xgboost,
    **config,
    scheduler=scheduler,
    verbose=1
)

# Get the best hyperparameters and the corresponding results
best_config = analysis.get_best_config(metric='mean_accuracy')
best_accuracy = analysis.best_result['mean_accuracy']

# Print the best hyperparameters and accuracy
print('Best hyperparameters:', best_config)
print('Best accuracy:', best_accuracy)

# Shutdown Ray
ray.shutdown()

ValueError: Tracked actor is not managed by this event manager: <TrackedActor 1036473497790429618054672036532613316>

In [18]:
import ray
from ray import tune
from ray.tune.schedulers import HyperBandScheduler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN


# Define your training function
def train_xgboost(config):
    # Load your dataset
    # Assuming you have features (X) and labels (y) as numpy arrays
#     X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
#     train_data = (ray.put(X_train), ray.put(y_train))  # Put large objects in Ray object store
#     test_data = (ray.put(X_test), ray.put(y_test))  # Put large objects in Ray object store

# Split the dataset into training and testing sets
    X_train, X_test_actual, y_train, y_test_actual = train_test_split(data_df.drop('predictor', axis=1), data_df['predictor'], test_size=0.2, random_state=42)


    # Use ADASYN to oversample the minority class
    adasyn = ADASYN(random_state=42)
    X_resampled, y_resampled = adasyn.fit_resample(X_train.iloc[:,1:], y_train)

    
    
    # Configure XGBoost with the given hyperparameters
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=config['n_estimators'],
        max_depth=config['max_depth'],
        learning_rate=config['learning_rate'],
        subsample=config['subsample'],
        # Add more hyperparameters as needed
    )

    # Train the model
    model.fit(X_resampled, y_resampled)

    # Evaluate the model
    accuracy = model.score(X_test_actual, y_test_actual)

    # Return the accuracy as the result
    return accuracy

# Define the hyperparameter search space
search_space = {
    'n_estimators': tune.choice([100, 200, 300]),
    'max_depth': tune.choice([3, 6, 9]),
    'learning_rate': tune.loguniform(0.001, 0.1),
    'subsample': tune.uniform(0.5, 1.0),
    # Add more hyperparameters and their search spaces as needed
}

# Initialize Ray
if not ray.is_initialized():
    ray.init(ignore_reinit_error=True)

# Define the hyperparameter search configuration
config = {
    'num_samples': 10,  # Number of trials
    'config': search_space,
    'stop': {'training_iteration': 100},  # Specify the stopping criteria
}

# Configure the scheduler (optional)
scheduler = HyperBandScheduler(
    time_attr='training_iteration',
    metric='mean_accuracy',
    mode='max'
    # max_t=100,
    # grace_period=10
)

# Run the hyperparameter search
analysis = tune.run(
    train_xgboost,
    **config,
    scheduler=scheduler,
    verbose=3
)

# Get the best hyperparameters and the corresponding results
best_config = analysis.get_best_config(metric='mean_accuracy')
best_accuracy = analysis.best_result['mean_accuracy']

# Print the best hyperparameters and accuracy
print('Best hyperparameters:', best_config)
print('Best accuracy:', best_accuracy)

# Shutdown Ray
ray.shutdown()

ValueError: Tracked actor is not managed by this event manager: <TrackedActor 198073294835160252171096984625556305212>

In [20]:
data_df

,account,ntf_cnt,inhome_cnt,wholehome_cnt,partialhome_cnt,drop_cnt,plant_cnt,high_cnt,medium_cnt,low_cnt,...,int_severity21,int_severity22,int_severity23,int_severity24,etds_past1yearcnt,download_ingb,upload_ingb,ecafs_cnt,num_devices,predictor
0,8347401110324147,89,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,252.80,58.05,0.0,1,0
1,8245112000167867,203,0,115,0,0,0,3,114,0,...,0,0,0,0,0.0,2401.08,215.49,0.0,1,0
2,8345781160019016,417,46,0,0,0,0,12,34,4,...,0,0,0,0,0.0,9770.24,393.00,0.0,1,0
3,8352305500262480,460,0,0,0,0,0,0,1,0,...,0,0,0,0,0.0,4288.84,809.74,0.0,1,0
4,8352305730035912,462,0,0,0,0,0,0,0,0,...,0,0,0,0,0.0,6346.49,1721.82,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209635,8348100022353876,457,0,0,0,0,9,9,0,0,...,0,0,0,0,0.0,930.29,19.10,0.0,1,0
209636,8348100023071022,325,117,0,16,0,6,76,65,0,...,0,0,0,0,0.0,159.95,11.91,0.0,1,0
209637,8348100023228077,414,45,0,6,0,1,19,2,31,...,1,0,0,27,0.0,4346.27,205.34,0.0,1,0
209638,8348100023288204,196,210,0,3,0,0,7,161,63,...,0,0,0,0,0.0,250.21,26.46,0.0,1,0


In [ ]:
import ray
from ray import tune
from ray.tune.schedulers import HyperBandScheduler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN

# Define your training function
def train_xgboost(config, X_train, y_train, X_test_actual, y_test_actual):
    # Use ADASYN to oversample the minority class
    adasyn = ADASYN(random_state=42)
    X_resampled, y_resampled = adasyn.fit_resample(X_train.iloc[:,1:], y_train)
    
    # Configure XGBoost with the given hyperparameters
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=config['n_estimators'],
        max_depth=config['max_depth'],
        learning_rate=config['learning_rate'],
        subsample=config['subsample'],
        # Add more hyperparameters as needed
    )

    # Train the model
    model.fit(X_resampled, y_resampled)

    # Evaluate the model
    accuracy = model.score(X_test_actual, y_test_actual)

    # Return the accuracy as the result
    return accuracy

# Define the hyperparameter search space
search_space = {
    'n_estimators': tune.choice([100, 200, 300]),
    'max_depth': tune.choice([3, 6, 9]),
    'learning_rate': tune.loguniform(0.001, 0.1),
    'subsample': tune.uniform(0.5, 1.0),
    # Add more hyperparameters and their search spaces as needed
}

# Initialize Ray
if not ray.is_initialized():
    ray.init(ignore_reinit_error=True)


# Split the dataset into training and testing sets
X_train, X_test_actual, y_train, y_test_actual = train_test_split(data_df.drop('predictor', axis=1), data_df['predictor'], test_size=0.2, random_state=42)

# Put large objects in Ray object store
X_train_id = ray.put(X_train.iloc[:,1:])
y_train_id = ray.put(y_train)

# Define the hyperparameter search configuration
config = {
    'num_samples': 10,  # Number of trials
    'config': search_space,
    'stop': {'training_iteration': 100},  # Specify the stopping criteria
}

# Configure the scheduler (optional)
scheduler = HyperBandScheduler(
    time_attr='training_iteration',
    metric='mean_accuracy',
    mode='max'
    # max_t=100,
    # grace_period=10
)

# Wrap the train_xgboost function with with_parameters and pass the large objects using ray.put()
train_xgboost_with_params = tune.with_parameters(train_xgboost, X_train=X_train_id, y_train=y_train_id, X_test_actual=X_test_actual.iloc[:,1:], y_test_actual=y_test_actual)

# Run the hyperparameter search
analysis = tune.run(
    train_xgboost_with_params,
    **config,
    scheduler=scheduler,
    verbose=3
)

# Get the best hyperparameters and the corresponding results
best_config = analysis.get_best_config(metric='mean_accuracy')
best_accuracy = analysis.best_result['mean_accuracy']

# Print the best hyperparameters and accuracy
print('Best hyperparameters:', best_config)
print('Best accuracy:', best_accuracy)

# Shutdown Ray
ray.shutdown()

In [22]:
import ray
from ray import tune
from ray.tune.schedulers import HyperBandScheduler
import xgboost as xgb
from sklearn.model_selection import train_test_split


ray.shutdown()
# Initialize Ray
if not ray.is_initialized():
    ray.init(ignore_reinit_error=True)

# Define your training function
def train_xgboost(config):
    # Load your dataset
    # Assuming you have features (X) and labels (y) as numpy arrays
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
    train_data = (ray.put(X_train), ray.put(y_train))  # Put large objects in Ray object store
    test_data = (ray.put(X_test), ray.put(y_test))  # Put large objects in Ray object store
    
    # Configure XGBoost with the given hyperparameters
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=config['n_estimators'],
        max_depth=config['max_depth'],
        learning_rate=config['learning_rate'],
        subsample=config['subsample'],
        # Add more hyperparameters as needed
    )

    # Train the model
    model.fit(train_data[0], train_data[1])

    # Evaluate the model
    accuracy = model.score(test_data[0], test_data[1])

    # Return the accuracy as the result
    return accuracy

# Define the hyperparameter search space
search_space = {
    'n_estimators': tune.choice([100, 200, 300]),
    'max_depth': tune.choice([3, 6, 9]),
    'learning_rate': tune.loguniform(0.001, 0.1),
    'subsample': tune.uniform(0.5, 1.0),
    # Add more hyperparameters and their search spaces as needed
}

# Define the hyperparameter search configuration
config = {
    'num_samples': 10,  # Number of trials
    'config': search_space,
    'stop': {'training_iteration': 100},  # Specify the stopping criteria
}

# Configure the scheduler (optional)
scheduler = HyperBandScheduler(
    time_attr='training_iteration',
    metric='mean_accuracy',
    mode='max'
)

# Run the hyperparameter search
analysis = tune.run(
    train_xgboost,
    **config,
    scheduler=scheduler,
    verbose=1
)

# Get the best hyperparameters and the corresponding results
best_config = analysis.get_best_config(metric='mean_accuracy')
best_accuracy = analysis.best_result['mean_accuracy']

# Print the best hyperparameters and accuracy
print('Best hyperparameters:', best_config)
print('Best accuracy:', best_accuracy)

# Shutdown Ray
ray.shutdown()

ValueError: Tracked actor is not managed by this event manager: <TrackedActor 123247576535370943055110005273167968738>